# CLOPS实验

CLOPS是由IBM提出的用于衡量量子计算机的运算速度的指标，全称为Circuit Layer Operations Per Second，即每秒量子线路层操作数，类似于传统计算机中的FLOPS（每秒浮点运算次数）用于衡量传统计算机的性能。

评价量子计算机性能有几个指标：质量、速度和规模。其中，量子计算机质量可以用量子体积(Quantum Volume)来表征，规模可以通过量子比特数目来定义，量子计算机的速度指标则是由CLOPS给出。

随着量子计算技术的不断发展，CLOPS成为了评估量子计算机性能和比较不同量子计算系统之间性能的重要指标。不同的量子计算机架构和实现方式会影响其CLOPS值，而高CLOPS值通常表示量子计算机在执行复杂计算任务时更为高效。

# 理论
CLOPS值通过下式计算：
 
 数学公式: $ CLOPS=\frac{M\times K \times S \times D}{\text{time\_taken}} $
 
 其中，数学公式: $ M $表示参数化量子线路模板的数量，数学公式: $ K $表示每一个参数化量子线路参数的更新次数，数学公式: $ S $表示shots次数，数学公式: $ D=\log_2(QV) $表示QV层数。

CLOPS实验主要由以下几个步骤组成：
1. 构建一组随机量子线路（QV或者clops_h线路）
2. 提交一组量子线路
3. 获取量子线路结果，并作为随机数种子生成下一组线路
4. 重复迭代，直到所有批次都计算完
5. 根据全部实验的运行时间和计算参数计算clops值

导入必要的包后，我们还需要确定CLOPS实验的配置参数，才能正确运行CLOPS实验，这些配置参数包括：总量子比特数，量子比特CZ结构（与量子比特总数对应），同时计算批量任务数目，测量次数，迭代次数和线路层数（仅CLOPS_H需要）等。

CLOPS实验的总体代码流程如下：

In [ ]:
import pyqpanda as pq

from originbench.util.chip_topology import CouplingMap
from originbench.clops_benchmark.clops_bench import CLOPSBench
from originbench.clops_benchmark.clops_options import CLOPSOptions, ComputationApproach

import requests
requests.packages.urllib3.disable_warnings()

#需要先从本源官网获取当前的用户api_token,需要确保有足够的算力资源
my_qcloud_token = "your api token"

def run_clops_benchmark(clops_options : CLOPSOptions):
    
    machine = pq.QCloud()
    machine.set_configure(72, 72)
    online_api_key = my_qcloud_token
    machine.init_qvm(online_api_key, False)

    machine.qAlloc_many(72)
    machine.cAlloc_many(72)

    print(clops_options)

    clops_bench = CLOPSBench(options=clops_options)

    clops_bench.run_clops_h(machine, write_log= False)
    # clops_bench.run_clops_v(machine, write_log= False)
    clops_bench.analyze()


if __name__=="__main__":
    
    cz_pattern_list = [
        [[(12, 13), (7, 8)]],
        [[(25, 31), (37, 36)]],
        [[(30, 31), (31, 37)]],
        [[(42, 48), (49, 50)]],
        [[(49, 55), (61, 67)]],
        [[(61, 67), (68, 69)]],
        [[(61, 62), (68, 69)]]
        ]

    coupling_list_sets = [
        [(12, 13), (13, 7), (7, 8)],
        [(25, 31), (31, 37), (37, 36)],
        [(30, 31), (31, 37), (37, 36)],
        [(42, 48), (48, 49), (49, 50)],
        [(49, 55), (55, 61), (61, 67)],
        [(61, 67), (67, 68), (68, 69)],
        [(61, 62), (62, 68), (68, 69)]
    ]
  
    coupling_map_index = 0
    cz_pattern = cz_pattern_list[coupling_map_index]
    coupling_list = coupling_list_sets[coupling_map_index]
    coupling_map = CouplingMap(cz_pattern = cz_pattern, couplinglist = coupling_list)

    # print(coupling_map)

    clops_options = CLOPSOptions()
    clops_options.set_options(max_batch_limits = 100,
                              circuits_num = 10,
                              layers_num = 5, # clops_h only
                              qubits_num = 4,
                              shots = 2000,
                              iterations = 1,
                              coupling_map = coupling_map,
                              chip_id = 72,
                              # specified_qubits_block = [45, 46],
                              is_amend = True,
                              is_mapping = False,
                              is_optimization = True)
    
    run_clops_benchmark(clops_options)

然后直接运行即可，其中CLOPS和CLOPS_H是两种测试线路构造，可任意选择

选择方式为修改run_clops_benchmark函数中的调用函数即可

1.CLOPS

基于QV线路的clops线路构造

**clops_bench.run_clops_v(machine, write_log= True)**

2.CLOPS_H
基于hardware-efficiency线路的clops线路构造

**clops_bench.run_clops_h(machine, write_log= True)**



所有计算的结果分析和参数配置信息均会输出到终端控制台上,如果开启clops_bench.run_clops_h(machine, write_log= True)的write_log选项，会在本地生成带有时间戳的log文件

# 参考文献
[1] https://research.ibm.com/blog/circuit-layer-operations-per-second (2023).